# Getting started with authentication and packages

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!earthengine authenticate
import ee
ee.Initialize()

In [ ]:
from ee import oauth
from google_auth_oauthlib.flow import Flow
import json

In [ ]:
!pip install rio-cogeo
!pip install rasterio
import glob
import rasterio
import os

In [ ]:
from pprint import pprint

# Define functions for uploading assets to GEE

In [ ]:
from google.auth.transport.requests import AuthorizedSession
ee.Authenticate()  
session = AuthorizedSession(ee.data.get_persistent_credentials())

In [ ]:
# Customize the following digits to make sure date format is correct based on the file name 
def requestBody(uris):
  assetname= uris.split("/")[-1][0:-4]
  year = assetname[-10:-6]  
  month = assetname[-5:-3]  
  day = assetname[-2:]  
  date= year +"-"+month+"-"+day  
  date= year +"-"+month+"-"+day + "T00:00:00.000000000Z"
  request = {
    'type': 'IMAGE',
    'gcs_location': {
      'uris':[uris]
    },
    'startTime': date
  }
  pprint(json.dumps(request))
  return request

In [ ]:
def sendRequest(request,asset_id):
  project_folder = 'earthengine-legacy'
  url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'
  response = session.post(
    url = url.format(project_folder, asset_id),
    data = json.dumps(request)
  )
  pprint(json.loads(response.content))

# Import files and create Cloud-optimized Tiffs

Generate field-specific layers

In [ ]:
# Customize field name and pathways and import the target depth layer(s)
!gsutil cp gs://bucket/Moisture/Field/5/*tif . 

In [ ]:
image_list = []
for files in glob.glob("*.tif"):
    image_list.append(files)
    image_name_nopath = os.path.basename(files)

In [ ]:
# Replace the original files with cloud optimized images
for file in image_list:
  filename = file
  !rio cogeo create $filename cog_$filename
  !gsutil cp cog_$filename gs://bucket/Moisture/Field/5/"$filename"

# Upload Cloud-optimized Tiffs to GEE

In [ ]:
# Customize the pathway to import files
files = !gsutil ls gs://bucket/Moisture/Field/5/*tif 

In [ ]:
for uris in files:
  name= uris.split("/")[-1][0:-4]
  assetname = "Field_5" + name[-11:] # Customize with depth
  assetpath="users/username/Moisture/Layers/Field/"+assetname # Customize with username and field name
  print(assetpath)
  thisrequest = requestBody(uris)
  sendRequest(thisrequest,assetpath)